# Import Required modules

In [33]:
import pandas as pd
import numpy as np

# Creating the Data cube( snowflake schema) 

In [34]:
dept_data = {
    "Dept_ID": [1,2,3],
    "Dept_Name": ["CSE","ECE","MECH"],
    "Dept_Dean": ["Dr.SS","Dr.AK","Dr.V"]
}
dept_df = pd.DataFrame(dept_data)
dept_df

,Dept_ID,Dept_Name,Dept_Dean
0,1,CSE,Dr.SS
1,2,ECE,Dr.AK
2,3,MECH,Dr.V


In [35]:
loc_data = {
    "Loc_ID": [1,2,3],
    "Loc_Name": ["SOC","VV","VKJ"]
}

loc_df = pd.DataFrame(loc_data)
loc_df

,Loc_ID,Loc_Name
0,1,SOC
1,2,VV
2,3,VKJ


In [36]:
year_data = {
    'year_ID': [1,2,3],
    'year': [2023,2024,2025]
}
year_df = pd.DataFrame(year_data)
year_df

,year_ID,year
0,1,2023
1,2,2024
2,3,2025


In [18]:
enroll_data = {
    "enroll_id": [1,2,3,4,5,6,7,8,9],
    "Dept_ID": [1,2,3,1,2,3,1,2,3],
    "Loc_ID": [1,2,3,1,2,3,1,2,3],
    "year_ID": [1,1,1,2,2,2,3,3,3],
    "enrollment_count": [500,550,300,600,550,320,550,555,310],
    "Faculty_count": [20,22,15,20,23,14,21,22,15]
}

enroll_df = pd.DataFrame(enroll_data)
enroll_df

,enroll_id,Dept_ID,Loc_ID,year_ID,enrollment_count,Faculty_count
0,1,1,1,1,500,20
1,2,2,2,1,550,22
2,3,3,3,1,300,15
3,4,1,1,2,600,20
4,5,2,2,2,550,23
5,6,3,3,2,320,14
6,7,1,1,3,550,21
7,8,2,2,3,555,22
8,9,3,3,3,310,15


-1)
-----------------------------------------------------------------------
# Slice on Year = 2024 

In [37]:
slice_df = enroll_df.merge(year_df, on='year_ID').query("year==2024")
print("Slice Operation Result:\n", slice_df)

Slice Operation Result:
    enroll_id  Dept_ID  Loc_ID  year_ID  enrollment_count  Faculty_count  year
3          4        1       1        2               600             20  2024
4          5        2       2        2               550             23  2024
5          6        3       3        2               320             14  2024


-2)
-----------------------------------------------------------------------
# Dice to extract small cube from cuboid based on the condition given

In [38]:
diced_df = enroll_df.merge(dept_df,on='Dept_ID').merge(loc_df,on='Loc_ID').query("Dept_Name=='CSE' and Loc_Name=='SOC'")
print("Diced Data for CSE at SOC:")
print(diced_df)

Diced Data for CSE at SOC:
   enroll_id  Dept_ID  Loc_ID  year_ID  enrollment_count  Faculty_count  \
0          1        1       1        1               500             20   
1          4        1       1        2               600             20   
2          7        1       1        3               550             21   

  Dept_Name Dept_Dean Loc_Name  
0       CSE     Dr.SS      SOC  
1       CSE     Dr.SS      SOC  
2       CSE     Dr.SS      SOC  


-3)
-----------------------------------------------------------------------
# Roll-up to combine two columns in the base cuboid

In [39]:
rollup_data = enroll_df.merge(loc_df,on='Loc_ID').merge(year_df,on='year_ID')
rollup_result = rollup_data.groupby('year_ID').agg({
    'enrollment_count': np.sum,
    'Faculty_count': np.sum
}).reset_index()
print("Roll-up Data:")
print(rollup_result)

Roll-up Data:
   year_ID  enrollment_count  Faculty_count
0        1              1350             57
1        2              1470             57
2        3              1415             58


-4)
-----------------------------------------------------------------------
# Drill-down to split particular attribute

In [41]:
drilldown_data = enroll_df.merge(loc_df,on='Loc_ID')
drilldown_result = drilldown_data.query("Loc_Name=='VKJ'")
print("drilldown_result:\n", drilldown_result)

drilldown_result:
    enroll_id  Dept_ID  Loc_ID  year_ID  enrollment_count  Faculty_count  \
6          3        3       3        1               300             15   
7          6        3       3        2               320             14   
8          9        3       3        3               310             15   

  Loc_Name  
6      VKJ  
7      VKJ  
8      VKJ  


-5)
-----------------------------------------------------------------------
# Pivot to view the cuboid in particular angle

In [42]:
pivot_data = enroll_df.merge(dept_df,on='Dept_ID')
pivot_result = pivot_data.pivot_table(index='Dept_Name',columns='year_ID',values='enrollment_count', aggfunc=np.sum, fill_value=0)
print("Pivoted Data - Year-wise Enrollment by Department:")
print(pivot_result)

Pivoted Data - Year-wise Enrollment by Department:
year_ID      1    2    3
Dept_Name               
CSE        500  600  550
ECE        550  550  555
MECH       300  320  310
